In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import pandas as pd
import subprocess
import os
from six.moves import urllib
import tempfile
import sys
import numpy as np
import re
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import strip_punctuation
from gensim.parsing.preprocessing import strip_multiple_whitespaces
from gensim.parsing.preprocessing import strip_non_alphanum
from unidecode import unidecode
import pickle
import spacy
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
# sp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

def strip_custom(token):
    token = token.replace('&Reg;', " ")
    token = token.replace("&lt;", "<")
    token = token.replace("&times;", "")
    token = token.replace("&gt;", ">")
    token = token.replace("&quot;", "")
    token = token.replace('&nbsp', " ")
    token = token.replace('&copy;', " ")
    token = token.replace('&reg', " ")
    token = token.replace('%20', " ")
    # this has to be last:
    token = token.replace("&amp;", "&")
    token = token.replace("â\x80¢", " ")
#     token = token.replace("Â®", " ")
#     token = token.replace("Ã©", " ")
#     token = token.replace("®", " ")
#     token = token.replace("©", " ")
#     token = token.replace("™", " ")
#     token = token.replace("•", "")
#     token = token.replace("�", "")
    
    token = token.replace("width:99pt", "")
    token = token.replace('class="xl66">', '')
    token = token.replace('&#160;', ' ')
    return token


def string_processor(token):
    str = unidecode(token)
    str = str.lower()
    
    str = strip_custom(str)
    str = remove_stopwords(str)
    str = remove_tags(str)
    str = strip_punctuation(str)
    str = strip_non_alphanum(str)
    str = strip_multiple_whitespaces(str)

    return str



In [2]:
def get_saved_token_and_Tosentence(corpus,savedToken):
#     corpus = df.description.tolist()


    # loading
    with open(savedToken, 'rb') as handle:
        tokenizer = pickle.load(handle)

#     tokenizer.fit_on_texts(corpus)
    sequences = tokenizer.texts_to_sequences(corpus)

    top_word = len(tokenizer.index_word) +1
    print("vocab:",top_word)
    return np.array(sequences)

def processing_sequences(x_test,maxlen=16):
    print(len(x_test), 'sample sequences')

    print('Average sample sequence length: {}'.format(
        np.mean(list(map(len, x_test)))))
    print('Pad sequences (samples x time)')
    x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

    print('X shape:', x_test.shape)
    return x_test

In [3]:
df = pd.read_csv('newtest.csv')

print("before drop: ",len(df))
df = df.dropna()
print("after drop: ",len(df))

df = df[['bucket_name','description','product_name']]
# to lowercase
df.bucket_name = df.bucket_name.apply(lambda x : x.lower())
df.description = df.description.apply(lambda x : string_processor(x))
df.product_name = df.product_name.apply(lambda x :string_processor(x))



before drop:  20665
after drop:  11066


In [4]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.classes_ = np.load('namesaved_encoder.npy',allow_pickle=True)
label = encoder.transform(df['bucket_name'])
N=len(encoder.classes_)
N

66

In [5]:
name_token = get_saved_token_and_Tosentence(df.product_name.tolist(),'nametokenizer.pickle')
des_token = get_saved_token_and_Tosentence(df.description.tolist(),'descriptiontokenizer.pickle')

X_name = processing_sequences(name_token,maxlen=16)
X_des = processing_sequences(des_token,maxlen=75)

vocab: 17305
vocab: 101235
11066 sample sequences
Average sample sequence length: 5.683625519609615
Pad sequences (samples x time)
X shape: (11066, 16)
11066 sample sequences
Average sample sequence length: 63.47026929333092
Pad sequences (samples x time)
X shape: (11066, 75)


In [6]:
import googleapiclient.discovery

def predict_json(project, model, instances, version=None):
    """Send json data to a deployed model for prediction.

    Args:
        project (str): project where the Cloud ML Engine Model is deployed.
        model (str): model name.
        instances ([Mapping[str: Any]]): Keys should be the names of Tensors
            your deployed model expects as inputs. Values should be datatypes
            convertible to Tensors, or (potentially nested) lists of datatypes
            convertible to tensors.
        version: str, version of the model to target.
    Returns:
        Mapping[str: any]: dictionary of prediction results defined by the
            model.
    """
    # Create the ML Engine service object.
    # To authenticate set the environment variable
    # GOOGLE_APPLICATION_CREDENTIALS=<path_to_service_account_file>
    service = googleapiclient.discovery.build('ml', 'v1')
    name = 'projects/{}/models/{}'.format(project, model)

    if version is not None:
        name += '/versions/{}'.format(version)

    response = service.projects().predict(
        name=name,
        body={'instances': instances}
    ).execute()

    if 'error' in response:
        raise RuntimeError(response['error'])

    return response['predictions']


# def get_true_label(encoder,y_hats):
#     labels = [np.argmax(np.array(i['dense'])) for i in y_hats]
#     return encoder.inverse_transform(labels),labels
    

In [7]:
def get_label(y_hat):
    return 


In [8]:
def get_batch_prediction(project, model, instances):
    instances = instances.tolist()
    y_hat = predict_json(project,model,instances)
    return np.array([np.array(i['dense']) for i in y_hat ])

def get_labels(project, model,batch_size,X):

    result_probs = []
    batches = list(range(0,len(X),batch_size))+[len(X)]
    for i in range(len(batches)-1):
        print('processing...',batches[i])
        probs = get_batch_prediction(project, model,X[batches[i]:batches[i+1]])
        result_probs.extend(probs)
    return np.array(result_probs)

In [9]:
%%time
y_hat_name = get_labels('groupby-development','FastTextNameGlove',10000,X_name)
y_hat_name_label =np.argmax(y_hat_name ,axis=1)

processing... 0
processing... 10000
CPU times: user 652 ms, sys: 72 ms, total: 724 ms
Wall time: 7.59 s


In [10]:
%%time
y_hat_des = get_labels('groupby-development','FastTextDesGlove',2000,X_des)
y_hat_des_label =np.argmax(y_hat_des ,axis=1)

processing... 0
processing... 2000
processing... 4000
processing... 6000
processing... 8000
processing... 10000
CPU times: user 904 ms, sys: 28 ms, total: 932 ms
Wall time: 15.9 s


In [11]:
print(classification_report(label, y_hat_name_label,digits=4))
print(classification_report(label, y_hat_des_label,digits=4))
print(classification_report(label, np.argmax(y_hat_des+y_hat_name ,axis=1),digits=4))


              precision    recall  f1-score   support

           0     0.9524    0.5263    0.6780        38
           2     0.8000    0.8000    0.8000        30
           3     0.1176    0.6667    0.2000         3
           4     0.8780    0.9863    0.9290       146
           5     0.7432    0.8209    0.7801       134
           6     0.5567    0.7248    0.6297       149
           7     0.0000    0.0000    0.0000        40
           8     0.5500    0.5500    0.5500        20
           9     0.6741    0.7982    0.7309       228
          10     0.9000    0.8182    0.8571        11
          13     0.8712    0.7808    0.8235       762
          14     0.8280    0.8442    0.8360       154
          15     0.9043    0.9130    0.9087       207
          16     0.9147    0.8881    0.9012       447
          17     0.6757    0.7426    0.7075       101
          18     0.9057    0.8303    0.8664       613
          19     0.2000    1.0000    0.3333         1
          20     0.0000    

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
